In [1]:
import numpy as np
import os
import cv2
import xml.etree.ElementTree as ET
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, ReLU, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.applications import Xception
from tensorflow.keras.utils import to_categorical

# Path to the folders containing images and annotations
annotations_folder = r'C:\Users\haris\AI\ML\Projects\Dog Breed Classification\annotations'
images_folder = r'C:\Users\haris\AI\ML\Projects\Dog Breed Classification\images'


In [2]:
# Initialize lists to store image data and labels
images = []
labels = []

# Parse XML annotations and load images
for breed_folder in os.listdir(annotations_folder):
    breed_path = os.path.join(annotations_folder, breed_folder)
    for annotation_file in os.listdir(breed_path):
        annotation_path = os.path.join(breed_path, annotation_file)
        
        tree = ET.parse(annotation_path)
        root = tree.getroot()
        
        breed = root.find('object/name').text
        
        img_path = os.path.join(images_folder, breed_folder, annotation_file + '.jpg')
        image = cv2.imread(img_path)
        image = cv2.resize(image, (224, 224))  # Resize to desired size
        image = image/255
        
        images.append(image)
        labels.append(breed)

# Convert labels to numerical format
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
num_classes = len(label_encoder.classes_)


In [3]:
# Split data into training, validation, and testing sets
X_train, X_temp, y_train, y_temp = train_test_split(images, labels_encoded, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Preprocess the labels
y_train_onehot = to_categorical(y_train, num_classes)
y_val_onehot = to_categorical(y_val, num_classes)
y_test_onehot = to_categorical(y_test, num_classes)

# Convert image lists to NumPy arrays
X_train = np.array(X_train)
X_val = np.array(X_val)
X_test = np.array(X_test)

# Build Simple CNN
def build_simple_cnn(input_shape, num_classes):
    model = Sequential()
    
    # Convolutional blocks
    for _ in range(4):
        model.add(Conv2D(64, (3, 3), padding='same', input_shape=input_shape))
        model.add(BatchNormalization())
        model.add(ReLU())
        model.add(MaxPooling2D((2, 2)))
        model.add(Dropout(0.25))
    
    # Flatten and fully connected layers
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    
    return model

# Build Xception model with transfer learning
def build_xception(input_shape, num_classes):
    base_model = Xception(weights='imagenet', include_top=False, input_shape=input_shape)
    for layer in base_model.layers:
        layer.trainable = False
    model = Sequential()
    model.add(base_model)
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))
    return model

# Specify input shape and number of classes
input_shape = (224, 224, 3)  # Example input shape
num_classes = 120  # Number of dog breeds


In [6]:
from tensorflow.keras import backend as K
K.clear_session()

In [7]:
# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train_onehot)).shuffle(len(X_train)).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val_onehot)).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test_onehot)).batch(batch_size)

# Build and compile Simple CNN model
simple_cnn_model = build_simple_cnn(input_shape, num_classes)
simple_cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Build and compile Xception model
xception_model = build_xception(input_shape, num_classes)
xception_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train Simple CNN
history_simple_cnn = simple_cnn_model.fit(train_dataset, epochs=10, validation_data=val_dataset)

# Train Xception
history_xception = xception_model.fit(train_dataset, epochs=10, validation_data=val_dataset)

# Evaluate models
simple_cnn_loss, simple_cnn_acc = simple_cnn_model.evaluate(test_dataset, verbose=2)
xception_loss, xception_acc = xception_model.evaluate(test_dataset, verbose=2)

print("Simple CNN - Test accuracy:", simple_cnn_acc)
print("Xception - Test accuracy:", xception_acc)

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.